In [ ]:
!pip install -U langchain langchain-community langchain-openai sentence-transformers faiss-cpu gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from google.colab import drive
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
transcript_folder = Path("/content/drive/MyDrive/ServiceNow_Audio_Transcripts")
transcript_folder.mkdir(parents=True, exist_ok=True)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

documents: list[Document] = []
for txt_path in tqdm(transcript_folder.glob("*.txt"), desc="Chunking transcripts"):
    raw_text = txt_path.read_text(encoding="utf-8")
    for chunk in splitter.split_text(raw_text):
        documents.append(
            Document(page_content=chunk, metadata={"source": txt_path.name})
        )

# 4. Summary
print(f"✅ Loaded and chunked {len(documents)} document chunks from {transcript_folder}.")

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_index = FAISS.load_local(
    faiss_dir,
    embedding_model,
    allow_dangerous_deserialization=True
)

# 4. Now you can query it or wrap it in a retriever/chain—no save_local() needed here
retriever = faiss_index.as_retriever(search_kwargs={"k": 5})

/tmp/ipython-input-14-1137632584.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

retriever = faiss_index.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
from typing import Tuple

def answer_question(question: str) -> Tuple[str, str]:
    """
    Runs the RetrievalQA chain and formats the answer along with source snippets.
    Returns:
      - answer text
      - formatted source list (one entry per document)
    """
    try:
        response = qa_chain(question)
        answer = response["result"]

        # Build source list with preview snippets
        sources = []
        for doc in response["source_documents"]:
            text = doc.page_content.strip()
            preview = text[:400] + "..." if len(text) > 400 else text
            sources.append(f"📄 {doc.metadata.get('source', 'unknown')}\n🔎 {preview}")

        return answer, "\n\n".join(sources)

    except Exception as e:
        err = f"Error: {e}"
        # Return the same error in both slots to display in UI
        return err, err


In [ ]:
code = '''import os
from typing import Tuple

import gradio as gr
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Securely load your API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY not set in environment")

# 2. Sample documents (replace with your FAISS index load in production)
documents = [
    Document(
        page_content="ServiceNow uses AI for incident classification.",
        metadata={"source": "intro.txt"}
    )
]

# 3. Build FAISS index (for production, load existing index instead)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(documents, embedding=embedding_model)

# 4. Initialize the OpenAI LLM
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

# 5. Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    return_source_documents=True
)

# 6. Define your Gradio interface function
def answer_question(question: str) -> Tuple[str, str]:
    """
    Returns:
      - The generated answer
      - A string of source filenames + snippet previews
    """
    response = qa_chain(question)
    answer = response["result"]

    sources = []
    for doc in response["source_documents"]:
        text = doc.page_content.strip()
        preview = text[:200] + "..." if len(text) > 200 else text
        sources.append(f"📄 {doc.metadata.get('source', 'unknown')}\n🔎 {preview}")

    return answer, "\n\n".join(sources)

gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a ServiceNow question..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources")
    ],
    title="ServiceNow QA Bot",
    description="A RAG-powered assistant over ServiceNow transcripts.",
    allow_flagging="never"
).launch()
'''

with open("app.py", "w") as f:
    f.write(code)

In [ ]:
requirements = '''gradio
langchain
langchain-openai
langchain-community
langchain-core
sentence-transformers
faiss-cpu
openai
yt-dlp
python-dotenv
'''

with open("requirements.txt", "w") as f:
    f.write(requirements)

In [ ]:
from google.colab import files
files.download("app.py")
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import gradio as gr

gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a ServiceNow question..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources")
    ],
    title="ServiceNow QA Assistant",
    description="RAG-powered assistant over your ServiceNow YouTube transcripts.",
    allow_flagging="never"
).launch(
    share=True,
    debug=True,
    inline=False,
    show_error=True,
    prevent_thread_lock=True
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2f4f3bf9b81474fc88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://2f4f3bf9b81474fc88.gradio.live
